In [1]:
from preprocess_mri import *
from nilearn.maskers import NiftiLabelsMasker
from nilearn.datasets import fetch_atlas_aal
from nilearn.image import load_img

In [2]:
n3_dir = os.path.join('data', 'mri', 'ADNI', '136_S_1227', 'MPR____N3__Scaled')
files = find_nifti_files(n3_dir)

mni_path = os.path.join('mni', 'icbm152.nii')
mni_img = load_img(mni_path)

aal = fetch_atlas_aal(version='3v2')
atlas_img = load_img(aal.maps)

for file in files:
    registered_img = register_to_mni(file, mni_img)
    masker, segmented = segment_with_atlas(registered_img, atlas_img, aal.labels)
    roi_vols = compute_roi_volumes(registered_img, atlas_img, aal.labels)
    print(f"\nFile: {file}")
    for name, vol in roi_vols.items():
        print(f"  {name}: {vol:.1f} mm³")

[fetch_atlas_aal] Dataset found in C:\Users\Dhruv Khanna\nilearn_data\aal_3v2


C:\Users\Dhruv Khanna\Documents\Coding stuff\ADML\preprocess_mri.py:21: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  registered_img = resample_to_img(img, mni_img, copy_header=True)
C:\Users\Dhruv Khanna\Documents\Coding stuff\ADML\preprocess_mri.py:21: UserWarning: The provided image has no sform in its header. Please check the provided file. Results may not be as expected.
  registered_img = resample_to_img(img, mni_img, copy_header=True)



File: data\mri\ADNI\136_S_1227\MPR____N3__Scaled\2007-02-21_13_12_42.0\I66824\ADNI_136_S_1227_MR_MPR____N3__Scaled_Br_20070810000731580_S26837_I66824.nii
  Background: 28208.0 mm³
  Precentral_L: 27048.0 mm³
  Precentral_R: 38960.0 mm³
  Frontal_Sup_2_L: 41008.0 mm³
  Frontal_Sup_2_R: 36056.0 mm³
  Frontal_Mid_2_L: 38880.0 mm³
  Frontal_Mid_2_R: 8304.0 mm³
  Frontal_Inf_Oper_L: 11192.0 mm³
  Frontal_Inf_Oper_R: 20232.0 mm³
  Frontal_Inf_Tri_L: 17208.0 mm³
  Frontal_Inf_Tri_R: 6512.0 mm³
  Frontal_Inf_Orb_2_L: 6992.0 mm³
  Frontal_Inf_Orb_2_R: 7904.0 mm³
  Rolandic_Oper_L: 10648.0 mm³
  Rolandic_Oper_R: 17176.0 mm³
  Supp_Motor_Area_L: 18968.0 mm³
  Supp_Motor_Area_R: 2328.0 mm³
  Olfactory_L: 2304.0 mm³
  Olfactory_R: 23936.0 mm³
  Frontal_Sup_Medial_L: 17072.0 mm³
  Frontal_Sup_Medial_R: 5752.0 mm³
  Frontal_Med_Orb_L: 6848.0 mm³
  Frontal_Med_Orb_R: 6816.0 mm³
  Rectus_L: 5960.0 mm³
  Rectus_R: 4400.0 mm³
  OFCmed_L: 4968.0 mm³
  OFCmed_R: 3544.0 mm³
  OFCant_L: 5184.0 mm³
  OFCant_

C:\Users\Dhruv Khanna\Documents\Coding stuff\ADML\preprocess_mri.py:21: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  registered_img = resample_to_img(img, mni_img, copy_header=True)
C:\Users\Dhruv Khanna\Documents\Coding stuff\ADML\preprocess_mri.py:21: UserWarning: The provided image has no sform in its header. Please check the provided file. Results may not be as expected.
  registered_img = resample_to_img(img, mni_img, copy_header=True)



File: data\mri\ADNI\136_S_1227\MPR____N3__Scaled\2009-03-23_11_59_29.0\I143856\ADNI_136_S_1227_MR_MPR____N3__Scaled_Br_20090514132202278_S64957_I143856.nii
  Background: 28208.0 mm³
  Precentral_L: 27048.0 mm³
  Precentral_R: 38960.0 mm³
  Frontal_Sup_2_L: 41008.0 mm³
  Frontal_Sup_2_R: 36056.0 mm³
  Frontal_Mid_2_L: 38880.0 mm³
  Frontal_Mid_2_R: 8304.0 mm³
  Frontal_Inf_Oper_L: 11192.0 mm³
  Frontal_Inf_Oper_R: 20232.0 mm³
  Frontal_Inf_Tri_L: 17208.0 mm³
  Frontal_Inf_Tri_R: 6512.0 mm³
  Frontal_Inf_Orb_2_L: 6992.0 mm³
  Frontal_Inf_Orb_2_R: 7904.0 mm³
  Rolandic_Oper_L: 10648.0 mm³
  Rolandic_Oper_R: 17176.0 mm³
  Supp_Motor_Area_L: 18968.0 mm³
  Supp_Motor_Area_R: 2328.0 mm³
  Olfactory_L: 2304.0 mm³
  Olfactory_R: 23936.0 mm³
  Frontal_Sup_Medial_L: 17072.0 mm³
  Frontal_Sup_Medial_R: 5752.0 mm³
  Frontal_Med_Orb_L: 6848.0 mm³
  Frontal_Med_Orb_R: 6816.0 mm³
  Rectus_L: 5960.0 mm³
  Rectus_R: 4400.0 mm³
  OFCmed_L: 4968.0 mm³
  OFCmed_R: 3544.0 mm³
  OFCant_L: 5184.0 mm³
  OFCan